<a href="https://colab.research.google.com/github/ruoxinli94/Python/blob/master/FBCommentDataXGbooster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install pydrive to load data
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
id = '1i7_nAEBlgaf6aZDV6_E7aMn0f63-J7L_'
file = drive.CreateFile({'id':id}) 
file.GetContentFile('Features_Variant_1.csv') 

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [5]:
df = pd.read_csv('Features_Variant_1.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,65,166,2,0,24,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,10,132,1,0,24,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,0,0,0,0,0,14,133,2,0,24,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,7,0,3,7,-3,62,131,1,0,24,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,806.0,7.574627,0.0,69.435826,0.0,76.0,2.604478,0.0,8.505502,0.0,806.0,10.649254,1.0,70.254788,-69.0,806.0,4.970149,0.0,69.85058,1,0,0,1,0,58,142,5,0,24,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [10]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53
count,4.094900e+04,40949.000000,4.094900e+04,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.0,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000,40949.000000
mean,1.313814e+06,4676.133752,4.480025e+04,24.254780,1.586241,443.333854,55.720384,35.645535,67.464151,0.219468,285.187428,22.186647,7.503724,40.474206,0.024103,268.358275,19.649686,4.921537,38.728848,1.497253,415.395297,52.631591,34.042370,63.154906,-220.046619,275.403722,2.536961,-2.020904,55.840996,55.720384,22.186647,19.649686,52.631591,2.536961,35.322035,163.652470,117.249823,0.0,23.767833,0.122347,0.143300,0.149747,0.157489,0.144033,0.146157,0.136926,0.141640,0.132506,0.137635,0.148599,0.150846,0.143886,0.144888,7.322889
std,6.785752e+06,20593.184863,1.109338e+05,19.950583,20.753174,496.695198,86.933548,69.960232,81.568249,10.055146,374.441728,36.930662,21.778756,54.277774,1.981360,327.063844,31.094112,13.245799,50.846434,18.715475,472.380251,81.264281,66.153081,76.403985,281.814185,373.330611,17.544907,14.720873,73.811134,136.975705,77.124263,71.078576,128.179920,94.202974,20.916864,376.264387,945.006667,0.0,1.919829,0.327691,0.350383,0.356828,0.364265,0.351127,0.353268,0.343774,0.348684,0.339045,0.344520,0.355698,0.357903,0.350979,0.351992,35.494550
min,3.600000e+01,0.000000,0.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1366.000000,-204.000000,-210.500000,-288.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1366.000000,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.673400e+04,0.000000,6.980000e+02,9.000000,0.000000,45.000000,5.527273,2.000000,8.278756,0.000000,26.000000,1.911290,0.000000,4.109465,0.000000,26.000000,2.030303,0.000000,4.094580,0.000000,42.000000,5.218182,2.000000,7.600215,-310.000000,23.000000,-0.483221,-2.000000,5.990950,2.000000,0.000000,0.000000,2.000000,-6.000000,17.000000,38.000000,2.000000,0.0,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.929110e+05,0.000000,7.045000e+03,18.000000,0.000000,241.000000,23.374101,12.000000,35.069140,0.000000,118.000000,8.437500,2.000000,17.382709,0.000000,116.000000,8.584270,1.000000,18.639984,0.000000,224.000000,21.859375,12.000000,32.368848,-92.000000,109.000000,0.273810,0.000000,25.547172,11.000000,2.000000,0.000000,10.000000,0.000000,35.000000,97.000000,13.000000,0.0,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.204214e+06,99.000000,5.026400e+04,32.000000,0.000000,717.000000,71.828829,42.000000,102.554954,0.000000,405.000000,29.005525,8.000000,60.760334,0.000000,381.000000,24.842520,5.000000,54.523165,0.000000,676.000000,67.913793,40.000000,96.266919,-21.000000,379.000000,2.974684,0.000000,81.209289,46.000000,12.000000,9.000000,44.000000,3.000000,53.000000,172.000000,61.000000,0.0,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,4

In [11]:
X,y = df.loc[:,:52].values, df.loc[:,53].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = .1, random_state = 42)

In [14]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [16]:
from sklearn.metrics import mean_absolute_error
# baseline model to compare
mean_train = np.mean(y_train)
baseline_pred = np.ones(y_test.shape)*mean_train
mae_baseline = mean_absolute_error(y_test, baseline_pred)
print('baseline MAE is {:.2f}'.format(mae_baseline))

baseline MAE is 11.31


In [29]:
# parameters need to tune
params = {
    'max_depth': 6,
    'min_child_weight': 1,
    'eta': .3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective': 'reg:squarederror',
}

- eta: Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative.
- gamma: Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
- max_depth: Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. 0 is only accepted in lossguided growing policy when tree_method is set as hist and it indicates no limit on depth. Beware that XGBoost aggressively consumes memory when training a deep tree.
- min_child_weight: Minimum sum of instance weight (hessian) needed in a child. If the tree partition step results in a leaf node with the sum of instance weight less than min_child_weight, then the building process will give up further partitioning. In linear regression task, this simply corresponds to minimum number of instances needed to be in each node. The larger min_child_weight is, the more conservative the algorithm will be.
- subsample: Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration.
- colsample_bytree:  is the subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.
- objective

<a>https://xgboost.readthedocs.io/en/latest/parameter.html</a>

In [27]:
params['eval_metric'] = "mae"
model = xgb.train(
    params,
    dtrain,
    num_boost_round=999,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:,.2f} with {} round".format(model.best_score, model.best_iteration+1))


[06:24:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	Test-mae:5.97475
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:5.03358
[2]	Test-mae:4.64571
[3]	Test-mae:4.42331
[4]	Test-mae:4.3933
[5]	Test-mae:4.35543
[6]	Test-mae:4.31317
[7]	Test-mae:4.33087
[8]	Test-mae:4.37164
[9]	Test-mae:4.38773
[10]	Test-mae:4.3944
[11]	Test-mae:4.4066
[12]	Test-mae:4.39125
[13]	Test-mae:4.39086
[14]	Test-mae:4.3983
[15]	Test-mae:4.39103
[16]	Test-mae:4.40307
Stopping. Best iteration:
[6]	Test-mae:4.31317

Best MAE: 4.31 with 7 round


In [28]:
cv = results = xgb.cv(
    params,
    dtrain,
    num_boost_round = 999,
    seed = 42,
    nfold = 5,
    metrics = {'mae'},
    early_stopping_rounds = 10
)
cv

[06:27:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:27:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,5.604926,0.064605,5.689178,0.270170
1,4.622580,0.065201,4.849516,0.271878
2,4.059671,0.065833,4.468350,0.239480
3,3.722956,0.060770,4.268607,0.224457
4,3.510318,0.061212,4.192441,0.189756
5,3.367232,0.061089,4.172880,0.189611
6,3.245605,0.060287,4.157868,0.192570
7,3.151511,0.062634,4.143261,0.194440
8,3.082250,0.059009,4.147865,0.196188
9,3.016822,0.057301,4.144676,0.189783


In [30]:
cv['test-mae-mean'].min()

4.0827864

In [35]:
# tune with gridsearchcv
from sklearn.model_selection import GridSearchCV
parameters = {
    'max_depth': [9,10,11],
    'min_child_weight': [5,6,7],
    'eta': [0.3],
}
Grid_XGBoost = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror', 
                                                        seed = 42, subsample = 1, colsample_bytree=1),
                            param_grid=parameters, scoring = 'neg_mean_absolute_error', verbose =0, n_jobs = 10, cv=5)
Grid_XGBoost.fit(X_train, y_train)
print_grid_search_metrics(Grid_XGBoost)

KeyboardInterrupt: ignored

In [37]:
# step tune 
num_boost_round = 999
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
]
# Define initial best params and MAE
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=9, min_child_weight=5
	MAE 4.0452416 for 6 rounds
CV with max_depth=9, min_child_weight=6
	MAE 4.0764706 for 5 rounds
CV with max_depth=9, min_child_weight=7
	MAE 4.0753826 for 5 rounds
CV with max_depth=10, min_child_weight=5
	MAE 4.0805926 for 5 rounds
CV with max_depth=10, min_child_weight=6
	MAE 4.035099199999999 for 5 rounds
CV with max_depth=10, min_child_weight=7
	MAE 4.087241799999999 for 5 rounds
CV with max_depth=11, min_child_weight=5
	MAE 4.0626312 for 5 rounds
CV with max_depth=11, min_child_weight=6
	MAE 4.0548336 for 5 rounds
CV with max_depth=11, min_child_weight=7
	MAE 4.0581054000000005 for 5 rounds
Best params: 10, 6, MAE: 4.035099199999999


In [38]:
params['max_depth'] = 10
params['min_child_weight'] = 6

In [39]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0
	MAE 4.035099199999999 for 5 rounds
CV with subsample=1.0, colsample=0.9
	MAE 4.098153 for 5 rounds
CV with subsample=1.0, colsample=0.8
	MAE 4.3478966 for 9 rounds
CV with subsample=1.0, colsample=0.7
	MAE 4.3637696 for 9 rounds
CV with subsample=0.9, colsample=1.0
	MAE 4.030343 for 5 rounds
CV with subsample=0.9, colsample=0.9
	MAE 4.0957296 for 5 rounds
CV with subsample=0.9, colsample=0.8
	MAE 4.393033 for 8 rounds
CV with subsample=0.9, colsample=0.7
	MAE 4.5614972 for 7 rounds
CV with subsample=0.8, colsample=1.0
	MAE 4.149043000000001 for 5 rounds
CV with subsample=0.8, colsample=0.9
	MAE 4.1802660000000005 for 6 rounds
CV with subsample=0.8, colsample=0.8
	MAE 4.3865124 for 8 rounds
CV with subsample=0.8, colsample=0.7
	MAE 4.542675000000001 for 8 rounds
CV with subsample=0.7, colsample=1.0
	MAE 4.0902732 for 5 rounds
CV with subsample=0.7, colsample=0.9
	MAE 4.174224000000001 for 6 rounds
CV with subsample=0.7, colsample=0.8
	MAE 4.457408 f

In [40]:
params['subsample'] = .9
params['colsample_bytree'] = 1.

In [45]:
%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgb.cv(params, dtrain, num_boost_round=num_boost_round, seed=42, nfold=5,  metrics=['mae'],early_stopping_rounds=10)
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs
CV with eta=0.3
CPU times: user 21.2 s, sys: 37.7 ms, total: 21.2 s
Wall time: 10.9 s
	MAE 4.030343 for 5 rounds

CV with eta=0.2
CPU times: user 27.3 s, sys: 43.5 ms, total: 27.4 s
Wall time: 14 s
	MAE 3.9869984 for 9 rounds

CV with eta=0.1
CPU times: user 46.3 s, sys: 63.2 ms, total: 46.3 s
Wall time: 23.7 s
	MAE 3.924081 for 22 rounds

CV with eta=0.05
CPU times: user 1min 17s, sys: 97.3 ms, total: 1min 17s
Wall time: 39.4 s
	MAE 3.8876182 for 44 rounds

CV with eta=0.01
CPU times: user 5min 27s, sys: 431 ms, total: 5min 27s
Wall time: 2min 46s
	MAE 3.864672 for 219 rounds

CV with eta=0.005
CPU times: user 10min 59s, sys: 799 ms, total: 11min
Wall time: 5min 35s
	MAE 3.8671116000000003 for 457 rounds

Best params: 0.01, MAE: 3.864672


In [50]:
params['eta'] = .01
params['eval_metric'] = 'mae'
params

{'colsample_bytree': 1.0,
 'eta': 0.01,
 'eval_metric': 'mae',
 'max_depth': 10,
 'min_child_weight': 6,
 'objective': 'reg:squarederror',
 'subsample': 0.9}

In [51]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[0]	Test-mae:7.68993
Will train until Test-mae hasn't improved in 10 rounds.
[1]	Test-mae:7.61914
[2]	Test-mae:7.54953
[3]	Test-mae:7.48279
[4]	Test-mae:7.41599
[5]	Test-mae:7.34896
[6]	Test-mae:7.28229
[7]	Test-mae:7.21566
[8]	Test-mae:7.14924
[9]	Test-mae:7.08897
[10]	Test-mae:7.02335
[11]	Test-mae:6.96141
[12]	Test-mae:6.90122
[13]	Test-mae:6.8396
[14]	Test-mae:6.78149
[15]	Test-mae:6.72352
[16]	Test-mae:6.66678
[17]	Test-mae:6.61227
[18]	Test-mae:6.55499
[19]	Test-mae:6.50536
[20]	Test-mae:6.45183
[21]	Test-mae:6.39647
[22]	Test-mae:6.34558
[23]	Test-mae:6.29567
[24]	Test-mae:6.25061
[25]	Test-mae:6.20208
[26]	Test-mae:6.15583
[27]	Test-mae:6.11226
[28]	Test-mae:6.06881
[29]	Test-mae:6.02299
[30]	Test-mae:5.97616
[31]	Test-mae:5.93378
[32]	Test-mae:5.89278
[33]	Test-mae:5.8511
[34]	Test-mae:5.812
[35]	Test-mae:5.77168
[36]	Test-mae:5.73085
[37]	Test-mae:5.69367
[38]	Test-mae:5.65465
[39]	Test-mae:5.61944
[40]	Test-mae:5.58296
[41]	Test-mae:5.54739
[42]	Test-mae:5.51158
[43]	Test-ma

In [52]:
num_boost_round = model.best_iteration + 1
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-mae:7.68993
[1]	Test-mae:7.61914
[2]	Test-mae:7.54953
[3]	Test-mae:7.48279
[4]	Test-mae:7.41599
[5]	Test-mae:7.34896
[6]	Test-mae:7.28229
[7]	Test-mae:7.21566
[8]	Test-mae:7.14924
[9]	Test-mae:7.08897
[10]	Test-mae:7.02335
[11]	Test-mae:6.96141
[12]	Test-mae:6.90122
[13]	Test-mae:6.8396
[14]	Test-mae:6.78149
[15]	Test-mae:6.72352
[16]	Test-mae:6.66678
[17]	Test-mae:6.61227
[18]	Test-mae:6.55499
[19]	Test-mae:6.50536
[20]	Test-mae:6.45183
[21]	Test-mae:6.39647
[22]	Test-mae:6.34558
[23]	Test-mae:6.29567
[24]	Test-mae:6.25061
[25]	Test-mae:6.20208
[26]	Test-mae:6.15583
[27]	Test-mae:6.11226
[28]	Test-mae:6.06881
[29]	Test-mae:6.02299
[30]	Test-mae:5.97616
[31]	Test-mae:5.93378
[32]	Test-mae:5.89278
[33]	Test-mae:5.8511
[34]	Test-mae:5.812
[35]	Test-mae:5.77168
[36]	Test-mae:5.73085
[37]	Test-mae:5.69367
[38]	Test-mae:5.65465
[39]	Test-mae:5.61944
[40]	Test-mae:5.58296
[41]	Test-mae:5.54739
[42]	Test-mae:5.51158
[43]	Test-mae:5.47646
[44]	Test-mae:5.4421
[45]	Test-mae:5.40779
[46

In [53]:
best_model.save_model("FBCommentDataXGboost.model")